# RooFit
<b>RooFit</b> is a OO analysis environment built on ROOT. It is essentially a collection of classes designed to augment root for data modeling whose aim is summarised below (shamelessly stolen from Wouter Verkerke)...

![](files/roofit.png)

We will cover a few of the basics in the session today but note there are many more tutorials available at this link: [https://root.cern.ch/root/html600/tutorials/roofit/index.html](https://root.cern.ch/root/html600/tutorials/roofit/index.html)

## RooFit objects

In Roofit, any variable, data point, function, PDF ... is represented by a c++ object

The most basic of these is the `RooRealVar`. Let's create one which will represent the mass of some hypothetical particle, we name it and give it an initial starting value and range.

In [1]:
RooRealVar MH("MH","mass of the Hypothetical Boson (H-boson) in GeV",125,120,130);
MH.Print();


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

RooRealVar::MH = 125  L(120 - 130) 


ok, great. This variable is now an object we can play around with. We can access this object and modify it's properties, such as its value. 

In [2]:
MH.setVal(130);
MH.getVal();
MH.Print();

RooRealVar::MH = 130  L(120 - 130) 


In particle detectors we typically don't observe this particle mass but usually define some observable which is *sensitive* to this mass. Lets assume we can detect and reconstruct the decay products of the H-boson and measure the invariant mass of those particles. We need to make another variable which represents that invariant mass

In [3]:
RooRealVar mass("m","m (GeV)",100,80,200);
mass.Print();

RooRealVar::m = 100  L(80 - 200) 


In the perfect world we would perfectly measure the exact mass of the particle in every single event. However, our detectors are usually far from perfect so there will be some resolution effect. Lets assume the resolution of our measurement of the invariant mass is 10 GeV and call it "sigma"

In [4]:
RooRealVar sigma("resolution","#sigma",10,0,20);
sigma.Print();

RooRealVar::resolution = 10  L(0 - 20) 


More exotic variables can be constructed out of these `RooRealVar`s using `RooFormulaVars`. For example, suppose we wanted to make a function out of the variables which represented the relative resolution as a function of the hypothetical mass MH. 

In [5]:
RooFormulaVar func("R","@0/@1",RooArgList(sigma,mass));
func.Print("v");

--- RooAbsArg ---
  Value State: DIRTY
  Shape State: DIRTY
  Attributes: 
  Address: 0x111565068
  Clients: 
  Servers: 
    (0x1109c17b0,V-) RooRealVar::resolution "#sigma"
    (0x1109c1278,V-) RooRealVar::m "m (GeV)"
  Proxies: 
    actualVars -> 
      1)  resolution
      2)           m
--- RooAbsReal ---

  Plot label is "R"
    --- RooFormula ---
    Formula: "@0/@1"
    (resolution,m)


Notice how there is a list of the variables we passed (the servers or "actual vars"). We can now plot the function. RooFit has a special plotting object `RooPlot` which keeps track of the objects (and their normalisations) which we want to draw. Since RooFit doesn't know the difference between which objects are/aren't dependant, we need to tell it. 

Right now, we have the relative resolution as $R(m,\sigma)$ whereas we want to plot $R_{\sigma}(m)$.

In [ ]:
TCanvas *can = new TCanvas();

//make the x-axis the "mass"
RooPlot *plot = mass.frame(); 
func.plotOn(plot);

plot->Draw();
can->Draw();

The main objects we are interested in using from RooFit are <b>"probability denisty functions"</b> or (PDFs). We can construct the PDF 

$$f(m|M_{H},\sigma)$$ 

as a simple Gaussian shape for example or a `RooGaussian` in RooFit language (think McDonald's logic, everything is a `RooSomethingOrOther`)

In [ ]:
RooGaussian gauss("gauss","f(m|M_{H},#sigma)",mass,MH,sigma);
gauss.Print("V");

Notice how the gaussian PDF, like the `RooFormulaVar` depends on our `RooRealVar` objects, these are its servers.  Its evaluation will depend on their values. 

The main difference between PDFs and Functions in RooFit is that PDFs are <b>automatically normalised to unitiy</b>, hence they represent a probability density, you don't need to normalise yourself. Lets plot it for the current values of $m$ and $\sigma$

In [ ]:
plot = mass.frame();
    
gauss.plotOn(plot);

plot->Draw();
can->Draw();

If we change the value of MH, the PDF gets updated at the same time.

In [ ]:
MH.setVal(125);
gauss.plotOn(plot,RooFit::LineColor(kRed));

MH.setVal(135);
gauss.plotOn(plot,RooFit::LineColor(kGreen));

plot->Draw();

can->Update();
can->Draw();

PDFs can be used to generate Monte Carlo data. One of the benefits of RooFit is that to do so only uses a single line of code!

As before, we have to tell RooFit which variables to generate in (e.g which are the observables for an experiment). In this case, each of our events will be a single value of "mass" $m$.

In [ ]:
RooDataSet *data = (RooDataSet*) gauss.generate(RooArgSet(mass),500); 
//The arguments are the set of observables, follwed by the number of events

data->Print();

Now we can plot the data as with other RooFit objects.

In [ ]:
plot = mass.frame();

data->plotOn(plot);
gauss.plotOn(plot);
gauss.paramOn(plot);

plot->Draw();
can->Update();
can->Draw();

Of course we're not in the business of generating MC events, but collecting <b>real data!</b>. Next we will look at using real data in RooFit...

## RooFit datasets

A dataset is essentially just a collection of points in N-dimensional (N-observables) space. There are two basic implementations in RooFit, 

1) an "unbinned" dataset - `RooDataSet`

2) a "binned" dataset - `RooDataHist`

both of these use the same basic structure as below

![](files/datastructure.png)

Lets create an empty dataset where the only observable, the mass. Points can be added to the dataset one by one ...

In [ ]:
RooDataSet mydata("dummy","My dummy dataset",RooArgSet(mass)); 
// We've made a dataset with one observable (mass)

mass.setVal(123.4);
mydata.add(RooArgSet(mass));
mass.setVal(145.2);
mydata.add(RooArgSet(mass));
mass.setVal(170.8);
mydata.add(RooArgSet(mass));


mydata.Print();

There are also other ways to manipulate datasets in this way as shown in the diagram below 

![](files/datasets_manip.png)


Luckily there are also Constructors for a `RooDataSet` from a `TTree` and for a `RooDataHist` from a `TH1` so its simple to convert from your usual ROOT objects.

Let's take an example dataset put together already.

In [ ]:
TFile *file = TFile::Open("tutorial.root");
file->ls();

Inside the file, there is something called a `RooWorkspace`. This is just the RooFit way of keeping a persistent link between the objects for a model. It is a very useful way to share data and PDFs/functions etc among CMS collaborators.

Let's take a look at it. It contains a `RooDataSet` and one variable. This time we called our variable (or observable) `CMS_hgg_mass`, let's assume now that this is the invariant mass of photon pairs where we assume our H-boson decays to photons.  

In [ ]:
RooWorkspace *wspace = (RooWorkspace*) file->Get("workspace");
wspace->Print("v");

Let's have a look at the data. The `RooWorkspace` has several accessor functions, we will use the `RooWorkspace::data` one. 

There are also `RooWorkspace::var`, `RooWorkspace::function` and `RooWorkspace::pdf` with (hopefully) obvious purposes.

In [ ]:
RooDataSet *hgg_data = (RooDataSet*) wspace->data("dataset");
RooRealVar *hgg_mass = (RooRealVar*) wspace->var("CMS_hgg_mass");

plot = hgg_mass->frame();

hgg_data->plotOn(plot,RooFit::Binning(160)); 
// Here we've picked a certain number of bins just for plotting purposes 

plot->Draw();
can->Update();
can->Draw();

## Likelihoods and Fitting to data 

The data we have in our file doesn't look like a Gaussian distribution. Instead, we could probably use something like an exponential to describe it. 

There is an exponential pdf already in RooFit (yep you guessed it) `RooExponential`. For a pdf, we only need one parameter which is the exponential slope $\alpha$ so our pdf is,  

$$ f(m|\alpha) = \dfrac{1}{N} e^{-\alpha m}$$

Where of course, $N = \int_{110}^{150} e^{-\alpha m} dm$ is the normalisation constant.

You can fund a bunch of available RooFit functions here: [https://root.cern.ch/root/html/ROOFIT_ROOFIT_Index.html](https://root.cern.ch/root/html/ROOFIT_ROOFIT_Index.html)

There is also support for a generic pdf in the form of a `RooGenericPdf`, check this link: [https://root.cern.ch/doc/v608/classRooGenericPdf.html](https://root.cern.ch/doc/v608/classRooGenericPdf.html)

In [ ]:
RooRealVar alpha("alpha","#alpha",-0.05,-0.2,0.01);
RooExponential expo("exp","exponential function",*hgg_mass,alpha);

We can use RooFit to tell us to estimate the value of $\alpha$ using this dataset. You will learn more about parameter estimation but for now we will just assume you know about maximising likelihoods. This <b>maximum likelihood estimator</b> is common in HEP and is known to give unbiased estimates for things like distribution means etc. 

This also introduces the other main use of PDFs in RooFit. They can be used to construct <b>likelihoods</b> easily.

The likelihood $\mathcal{L}$ is defined for a particluar dataset (and model) as being proportional to the probability to observe the data assuming some pdf. For our data, the probability to observe an event with a value in an interval bounded by $a$ and $b$ is given by,

$$ P\left(m~\epsilon~[a,b] \right) = \int_{a}^{b} f(m|\alpha)dm  $$

As that interval shrinks we can say this probability just becomes equal to $f(m|\alpha)dm$

The probability to observe the dataset we have is given by the product of such probabilities for each of our data points, so that 

$$\mathcal{L}(\alpha) \propto \prod_{i} f(m_{i}|\alpha)$$

Note that for a specific dataset, the $dm$ factors which should be there are constnant. They can therefore be absorbed into the constant of proportionality!

The maximum likelihood esitmator for $\alpha$, usually written as $\hat{\alpha}$, is found by maximising $L(\alpha)$.

Note that this won't depend on the value of the constant of proportionality so we can ignore it. This is true in most scenarios because usually only the <b>ratio</b> of likelihoods is needed, in which the constant factors out. 

Obviously this multiplication of exponentials can lead to very large (or very small) numbers which can lead to numerical instabilities. To avoid this, we can take logs of the likelihood. Its also common to multiply this by -1 and minimize the resulting <b>N</b>egative <b>L</b>og <b>L</b>ikelihood</b> : $-Log \mathcal{L}(\alpha)$.

RooFit can construct the <b>NLL</b> for us.

In [ ]:
RooNLLVar *nll = (RooNLLVar*) expo.createNLL(*hgg_data);
nll->Print("v");

Notice that the NLL object knows which RooRealVar is the parameter because it doesn't find that one in the dataset. This is how RooFit distiguishes between <it>observables</it> and <it>parameters</it>.

RooFit has an interface to Minuit via the `RooMinimizer` class which takes the NLL as an argument. To minimize, we just call the `RooMinimizer::minimize()` function. "Minuit" is the program and "migrad" is the minimization routine which uses gradient descent...

In [ ]:
RooMinimizer minim(*nll);
minim.minimize("Minuit","migrad");  

RooFit has found the best fit value of alpha for this dataset. It also estimates an uncertainty on alpha using the Hessian matrix from the fit.

In [ ]:
alpha.Print("v");

Lets plot the resulting exponential on the data. Notice that the value of $\hat{\alpha}$ is used for the exponential. 

In [ ]:
expo.plotOn(plot);
expo.paramOn(plot);
plot->Draw();
can->Update();
can->Draw();

It looks like there could be a small region near 125 GeV for which our fit doesn't quite go through the points. Maybe our hypothetical H-boson isn't so hypothetical after all!

Let's see what happens if we include some resonant signal into the fit. We can take our Gaussian function again and use that as a signal model. A reasonable value for the resolution of a resonant signal with a mass around 125 GeV decaying to a pair of photons is around a GeV.

In [ ]:
sigma.setVal(1.);
sigma.setConstant();

MH.setVal(125);
MH.setConstant();

RooGaussian hgg_signal("signal","Gaussian PDF",*hgg_mass,MH,sigma);

By setting these parameters constant, RooFit knows (either when creating the NLL by hand or when using `fitTo`) that there is not need to fit for these parameters. 

We need to add this to our exponential model and fit a "Sigmal+Background model" by creating a `RooAddPdf`. In RooFit there are two ways to add PDFs, recursively where the fraction of yields for the signal and background is a parameter or absolutely where each PDF has its own normalisation. We're going to use the second one.

In [ ]:
RooRealVar norm_s("norm_s","N_{s}",10,100);
RooRealVar norm_b("norm_b","N_{b}",0,1000);

const RooArgList components(hgg_signal,expo);
const RooArgList coeffs(norm_s,norm_b);

RooAddPdf model("model","f_{s+b}",components,coeffs);
model.Print("v");

Ok now lets fit the model. Note this time we add the option `Extended()` which tells RooFit that we care about the overall number of observed events in the data $n$ too. It will add an additional Poisson term in the likelihood to account for this so our likelihood this time looks like,

$$L_{s+b}(N_{s},N_{b},\alpha) = \dfrac{ N_{s}+N_{b}^{n} e^{N_{s}+N_{b}} }{n!} \cdot \prod_{i}^{n} \left[ c f_{s}(m_{i}|M_{H},\sigma)+ (1-c)f_{b}(m_{i}|\alpha)  \right] $$

where $c = \dfrac{ N_{s} }{ N_{s} + N_{b} }$,   $f_{s}(m|M_{H},\sigma)$ is the Gaussian signal pdf and $f_{b}(m|\alpha)$ is the exponential pdf. Remember that $M_{H}$ and $\sigma$ are fixed so that they are no longer parameters of the likelihood.

There is a simpler interface for maximum likelihood fits which is the `RooAbsPdf::fitTo` method. With this simple method, RooFit will construct the negative log-likelihood function, from the pdf, and minimize all of the free parameters in one step...

In [ ]:
model.fitTo(*hgg_data,RooFit::Extended());

In [ ]:
model.plotOn(plot,RooFit::Components("exp"),RooFit::LineColor(kGreen));
model.plotOn(plot,RooFit::LineColor(kRed));
model.paramOn(plot);

can->Clear();
plot->Draw();
can->Update();
can->Draw();

What about if we also fit for the mass ($M_{H}$)? we can easily do this by removing the constant setting on MH...

In [ ]:
MH.setConstant(false);
model.fitTo(*hgg_data,RooFit::Extended());

Notice now the result for the fitted MH is not 125 and gets added to the fitted parameters since now it is floating.

We can get more information about the fit, via the `RooFitResult`, using the option `Save()`. 

In [ ]:
RooFitResult *fit_res = (RooFitResult*) model.fitTo(*hgg_data,RooFit::Extended(),RooFit::Save());
fit_res->Print("v");

For example, we can get the Correlation Matrix from the fit result... Note that the order of the parameters are the same as listed in the "Floating Parameter" list above

In [ ]:
TMatrixDSym cormat = fit_res->correlationMatrix();
cormat.Print();

And there even some fancy ways to visualise these results ... 

In [ ]:
TCanvas *can2 = new TCanvas("c","c",1000,460);
can2->Divide(2);

can2->cd(1);

RooPlot plot_err_mat(norm_b,norm_s,123,126,0,80);
fit_res->plotOn(&plot_err_mat,MH,norm_s,"MEVH12");
plot_err_mat.Draw();

can2->cd(2);

plot = hgg_mass->frame();
model.plotOn(plot,RooFit::VisualizeError(*fit_res,1),RooFit::FillColor(kOrange));
model.plotOn(plot);
plot->Draw();

can2->Draw();

A nice feature of RooFit is that once we have a pdf, data and results like this, we can import this new model into our `RooWorkspace` and show off our new discovery to our LHC friends (if we weren't about 5 years too late!). We can also save the "state" of our parameters for later, by creating a snapshot of the current values. 

In [ ]:
wspace->import(model);  
RooArgSet *params = model.getParameters(*hgg_data);
wspace->saveSnapshot("nominal_values",*params);

wspace->Print("V");

# Removing Nuisance Parameters

In HEP, we often have the case where there are some parameters of the model which are physics parameters of interest, while others are known as nuisance parameters. In some cases, those parameters may have external constraints, coming from other data or some theoretical reasoning. This is how we often treat <b>systematic uncertainties</b> but we won't cover those today.

There are two schools of thought for removing nuisance parameters 

   * Frequentists use profiling 
   * Bayesians use marginalisation

In our case, the hypothesis mass $M_{H}$ might be the thing which we can write papers on and win Nobel prizes for while we're not nearly so interested in the values of $N_{s}$, $\alpha$ or $N_{b}$. 

For simplicity, we will assume that the values of $\alpha$ and $N_{b}$ are fixed (maybe from theory or from a large Monte Carlo simulation) and so the only remaining nuisance paraemeter is $N_{s}$. 

In [ ]:
alpha.setConstant(true);
norm_b.setConstant(true);

The <b>profile likelihood</b> is one which removes the nuisances parameters by fitting them away. Note this is a major divide for Bayesians and Freqeuentist since removing nuisance parameters for Bayesians involves integrating (or marginalising) over them. 

The profile likeihood is written as, 

$$\mathcal{L}_{p}(M_{H})= \mathcal{L}_{s+b}(M_{H},\hat{\hat{N}}_{s})$$

where the double hat notation, $\hat{\hat{\cdot}}$, denotes the values of the nuisance parameter which maximises  $L_{s+b}$ at a given value of $M_{H}$.

RooFit has a useful class under the RooStats libraries for profile likelihoods: `ProfileLikelihoodCalculator` : [https://root.cern.ch/doc/v606/classRooStats_1_1ProfileLikelihoodCalculator.html](https://root.cern.ch/doc/v606/classRooStats_1_1ProfileLikelihoodCalculator.html)

In [ ]:
MH.setRange(123,126);
RooStats::ProfileLikelihoodCalculator plc(*hgg_data,model,RooArgSet(MH));

According to Wilks' theorem, -2 times the log of profile likelihood ratio, 

$$-2\log \lambda = -2\log \dfrac{\mathcal{L}(M_{H})}{\mathcal{L}(\hat{M}_{H})}$$

should be distributed as a $\chi^{2}$ with the number degrees of freedom equal to the number of parameters (in our case 1). This means that we can obtain the $1\sigma$ (or 68%) uncertainty by finding the interval for which 

$$-\log \lambda(M_{H})< 0.5 $$

We can tell RooFit to find that interval for us, 

In [ ]:
plc.SetConfidenceLevel(0.68);
RooStats::LikelihoodInterval *interval = plc.GetInterval();

std::cout << "\n\t hat{MH} = " 
    << ((RooRealVar*)interval->GetBestFitParameters()->find("MH"))->getVal() 
    << std::endl;

std::cout << "\t 68% interval = (" 
    << interval->LowerLimit(MH) << "," << interval->UpperLimit(MH) << ")" 
    << std::endl;

We can see where these values come from by drawing the negative of the profile likelihood ratio.

In [ ]:
RooStats::LikelihoodIntervalPlot plotLI(interval);

TCanvas *canLI = new TCanvas(); 
plotLI.Draw();
canLI->Draw();

We can also obtain a Bayesian <b>posterior</b> density by using Bayes' theorem ...

$$ P(M_{H}|\mathrm{data}) = \frac{ \int P(\mathrm{data}|M_{H},N_{s}) \pi(N_{s}|M_{H}) dN_{s}}{P(\mathrm{data})}$$

where $\pi(N_{s}|M_{H})$ is the (condidtional) <b>prior</b> probability density for our parameters. You might notice that  $P(\mathrm{data}|M_{H},N_{s})$ is nothing more than our likelihood function $\mathcal{L}(M_{H},N_{s})$!

Notice that I can use "$=$" here because we normalise according to $P(\mathrm{data})$, which is the integral over the likelihood (multiplied by the prior) with respect to <b>all</b> of its parameters. 

We can get RooFit to calculate this posterior using the `BayesianCalculator` class but first, for this time, we are going to create a `ModelConfig` which keeps track of which parameters are POIs and which are nuisances/observables. This is useful when sharing your model between collegues. 

In [ ]:
RooStats::ModelConfig mconfig("mymodel");
mconfig.SetWorkspace(*wspace);

mconfig.SetPdf("model");
mconfig.SetParametersOfInterest("MH");
mconfig.SetObservables("CMS_hgg_mass");
mconfig.SetNuisanceParameters("norm_s");

There are a few things to set up since we're using variables inside the workspace. First, lets set our two background  parameters to constant, this time, directly in the workspace


In [ ]:
wspace->var("norm_b")->setConstant(true);
wspace->var("alpha")->setConstant(true);

And we will load the nominal values from the fit before...

In [ ]:
wspace->loadSnapshot("nominal_values");
wspace->allVars().Print("V");

For Bayesian results, we need to set the prior density function ($\pi$). For now, let's assume no prior knowledge of the parameters - i.e the prior is constant within the range of the parameters and 0 elsewhere. 

Note that choosing a suitable prior is a debated subject amongst Bayesians since the results can depend on what you choose as a prior. Worse still, if a flat prior is chosen, say for $M_{H}$, should it be flat in $M_{H}$, or flat in $M_{H}^{2}$ (eg since $M_{H}=2\lambda v^{2}$ and maybe $\lambda$ is the fundamental thing)?

In [ ]:
wspace->var("MH")->setRange(123,126);

RooUniform flat_p("flat_prior","flat_prior",RooArgSet(MH,norm_s));
wspace->import(flat_p);
mconfig.SetPriorPdf("flat_prior");

mconfig.Print("v");

Now we can create the calculator and plot the posterior pdf...

In [ ]:
RooStats::BayesianCalculator bCalculator(*hgg_data,mconfig);
bCalculator.SetConfidenceLevel(0.68);
bCalculator.Print("v");
RooPlot *plotBC = bCalculator.GetPosteriorPlot();

In [ ]:
TCanvas *cBC = new TCanvas();
plotBC->Draw();
cBC->Draw();

This was the "shortest" interval, but we can also find the other valid intervals $[a,b]$ which satisfy $P(M_{H}~\epsilon~[a,b]) =0.68$. Let's compare the Bayesian 68% credible interval with our 65% confidence interval from the profile likelihood scan...

In [ ]:
RooStats::SimpleInterval *intervalBayes 
    = (RooStats::SimpleInterval*)bCalculator.GetInterval();

std::cout << "\n\t 68% (Bayes shortest) interval \t= (" 
    << intervalBayes->LowerLimit() << "," << intervalBayes->UpperLimit() << ")" 
    << std::endl;

std::cout << "\t 68% (frequentist) interval \t= (" 
    << interval->LowerLimit(MH) << "," << interval->UpperLimit(MH) << ")" 
    << std::endl;

They agree pretty well for our analysis but beware that this is not always the case!

You will learn a more about Likelihoods and their use in Hypothesis Testing (Frequentist/Basyesian intervals, coverage and credibility, setting limits, discovery...) in future statistics Lectures. There are many more useful toos for performing them with the <b>RooStats</b> library [https://twiki.cern.ch/twiki/bin/view/RooStats/WebHome](https://twiki.cern.ch/twiki/bin/view/RooStats/WebHome) - check them out!